<h1>Encriptado de imsis</h1>
<h2>Librerias</h2>
<p>En primer lugar, importamos las librerias necesarias.</p>

In [ ]:
import numpy as np #Libreria matemática
import hashlib #libreria de hashes
import json #Libreria para dumpear diccionarios
import time #Libreria para calcular cuanto le cuesta hacer las cosas
import socket #Para la conexión con el servidor
# Las librerias de abajo son las librerias para encriptar el mensaje antes de mandarlo
import cryptography as crypt
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization

<h2>Generación de imsi aleatorio</h2>
<p>Ahora vamos a generar un imsi aleatorio. Para eso veamos la estructura del imsi. Son 3 digitos del MCC (Código del operador), luego 2 o 3 dígitos del MNC (Código del pais) y luego 10 digitos del MSIN (que es la parte que nos identifica como usuarios.</p>


In [2]:
listadigitos = np.random.randint(0,10,size=(15,1)).flatten()
IMSI = []
for i in listadigitos:
    IMSI.append(str(i))
IMSI = "".join(IMSI)
print("El imsi generado es",IMSI)

El imsi generado es 151452598322403


<p>Para que sea facil crear IMSIs, creamos una función.<p>

In [3]:
def generarIMSI():
    listadigitos = np.random.randint(0,10,size=(15,1)).flatten()
    IMSI = []
    for i in listadigitos:
        IMSI.append(str(i))
    IMSI = "".join(IMSI)
    return IMSI

<h2>Procesado del IMSI</h2>
<p>Para procesar el IMSI, vamos a aplicarle un hash a este (que es una función inyectiva. Esto lo vamos a hacer después de sacar el código del Pais y del Operador.</p>

In [4]:
def procesarIMSI(imsi):
    procesado = {}
    procesado["MCC"] = imsi[:3] #El MCC son los 3 primeros dígitos
    procesado["MNC"] = imsi[3:5]
    procesado["Hash"] = hashlib.sha3_256(imsi.encode("ASCII")).hexdigest() #Creamos un hash con sha3_256 que es muy seguro
    return procesado

<h2>Encriptado del mensaje enviado</h2>
<p>Ahora vamos a encriptar el mensaje que se manda al servidor. Para eso vamos a generar en primer lugar una clave pública y una clave privada.</p>

In [ ]:
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
public_key = private_key.public_key()

<p>Vamos a necesitar la clave privada en el servidor así que hacemos una función para guardar la clave y otra para leerla.</p>

In [5]:
def guardarClavePrivada(private_key,nombrearchivo):
    pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    with open(nombrearchivo, 'wb') as f:
        f.write(pem)

In [6]:
def leerClavePrivada(nombrearchivo):
    with open(nombrearchivo, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend()
        )
    return private_key

<p>Ahora hacemos lo mismo con las claves públicas, ya que necesitamos la clave pública asociada a la clave privada.</p>

In [7]:
def guardarClavePublica(public_key,nombrearchivo):
    pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    with open(nombrearchivo, 'wb') as f:
        f.write(pem)

In [8]:
def leerClavePublica(nombrearchivo):
    with open(nombrearchivo, "rb") as key_file:
        public_key = serialization.load_pem_public_key(
            key_file.read(),
            backend=default_backend()
        )
    return public_key

<p>A continuación, probamos a guardar y a leer las claves publicas y privadas. Primero, guardamos las claves.</p>

In [9]:
guardarClavePublica(public_key,"public_key.pem")
guardarClavePrivada(private_key,"private_key.pem")

NameError: name 'public_key' is not defined

<p>Ahora, leemos las claves de los archivos que hemos guardado.</p>

In [10]:
public_key = leerClavePublica("public_key.pem")
private_key = leerClavePrivada("private_key.pem")

<p>Ahora creamos una función para encriptar y otra para desencriptar.</p>

In [11]:
def encriptar(public_key,message):
    encrypted = public_key.encrypt(
        message.encode("ASCII"),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return encrypted

In [12]:
def desencriptar(private_key,encrypted):
    message = private_key.decrypt(
        encrypted.encode("ASCII"),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return message.decode("ASCII")

<h2>Envio del dato encriptado</h2>
<p>Vamos a crear una función que envie a través de un socket los datos encriptados. Para eso vamos a establecer un pequeño protocolo:
<ul>
    <li>Para comenzar la conexión pondremos START CONECTION. El servidor contestará CONECTION STARTED o ERROR.</li>
    <li>Para enviar un diccionario de imsi encriptado pondremos IMSI diccionario. El servidor contestará OK o ERROR.</li>
    <li>Para cerrar la conexión enviaremos STOP CONECTION. El servidor no contestará.</li>
</ul>
</p>

In [13]:
class ComunicationError(ValueError):
    def __init__(self, message, *args):         
        super(ComunicationError, self).__init__(message, *args)


In [14]:
def enviarIMSIs(listaDeDiccionarios,HOST,PORT):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as mi_socket:
        tupla = (HOST, int(PORT))
        
        #Defino el protocolo
        protocolo = {}
        protocolo["INICIO_SEND"] = "START CONECTION"
        protocolo["INICIO_RECEIVE"] = "CONECTION STARTED"
        protocolo["IMSI_SEND"] = "IMSI "
        protocolo["IMSI_RECEIVE"] = "OK"
        protocolo["FIN_SEND"] = "STOP CONECTION"
        
        mi_socket.connect(tupla)
        mi_socket.sendall(bytes(protocolo["INICIO_SEND"], encoding='utf8'))
        
        dato_recibido_en_bytes = mi_socket.recv(1024)
        dato_recibido_en_str = dato_recibido_en_bytes.decode("utf-8")
        if dato_recibido_en_str[:len(protocolo["INICIO_RECEIVE"])] == protocolo["INICIO_RECEIVE"]:
            print("Comunicación iniciada")
        else:
            raise ComunicationError("Ha habido un error al iniciar la comunicación con el protocolo.")
        for i in listaDeDiccionarios:
            mi_socket.sendall(bytes(protocolo["IMSI_SEND"]+str(i), encoding='utf8'))
            dato_recibido_en_bytes = mi_socket.recv(1024)
            dato_recibido_en_str = dato_recibido_en_bytes.decode("utf-8")
            if dato_recibido_en_str[:len(protocolo["IMSI_RECEIVE"])] == protocolo["IMSI_RECEIVE"]:
                print("Diccionario enviado")
            else:
                raise ComunicationError("Ha habido un error al enviar un diccionario.")
        mi_socket.sendall(bytes(protocolo["FIN_SEND"], encoding='utf8'))
        time.sleep(1)

<h2>Prueba final</h2>

In [22]:
listaaenviar = []
for i in range(100):
    imsi = generarIMSI()
    procesado = procesarIMSI(imsi)
    encriptada = encriptar(public_key,json.dumps(procesado))
    listaaenviar.append(encriptada)


In [23]:
enviar = True
while enviar:
    try:
        enviarIMSIs(listaaenviar,"localhost","9999")
        break
    except:
        print("Lo intento de nuevo")

Comunicación iniciada
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviado
Diccionario enviad